# Correlación Precio - Sentimiento
## Librerías a utilizar: 

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay


## Datos a usar

In [8]:
eurusd = pd.read_csv("../data/EUR_USD_prices.csv")
eurusd["date"] = pd.to_datetime(eurusd["date"])
eurusd.set_index("date", inplace=True)

sent = pd.read_csv("../data/daily_sentiment.csv")  # <- archivo del notebook 2
sent["date"] = pd.to_datetime(sent["date"])
sent.set_index("date", inplace=True)

# promedio diario de sentimiento con ffill
sent_daily = sent.resample("D").mean().ffill()

# merge
df = eurusd.join(sent_daily, how="inner")


## Features

In [9]:
df["ret"] = np.log(df["4. close"] / df["4. close"].shift(1))
df["dir"] = (df["ret"] > 0).astype(int)
df["tr"] = df["2. high"] - df["3. low"]
df["vol20"] = df["ret"].rolling(20).std()

for lag in [1,2,3,5]:
    df[f"sent_lag{lag}"] = df["sent_mean"].shift(lag)

df.dropna(inplace=True)

# target = dirección del día siguiente
df["target"] = df["dir"].shift(-1)
df.dropna(inplace=True)
